In [741]:
# import Selenium module & other modules
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from html import unescape
from bs4 import BeautifulSoup
import requests
import time
import re
import pandas as pd

In [742]:
# open ALDI website
options = Options()
options.page_load_strategy = 'normal'
driver = webdriver.Chrome(options=options)
driver.get("https://new.aldi.us/")

#driver.implicitly_wait(3) 
#object of ActionChains
a = ActionChains(driver)
driver.set_window_size(880, 1080)

In [761]:
def check_xpath_exist(xpath,true_returning,false_returning):
    try:
        driver.find_element(By.XPATH,xpath)
    except NoSuchElementException:
        try: 
            time.sleep(10)
            driver.find_element(By.XPATH,xpath)
        except NoSuchElementException:
            return false_returning
        
    if true_returning == 'text':
        return driver.find_element(By.XPATH,xpath).text
    else:
        return true_returning

In [752]:
def ignore_error_when_click(element):
    try:
        element.click()
    except StaleElementReferenceException as e1:
        pass
    except ElementNotInteractableException as e2:
        pass
    except ElementClickInterceptedException as e3:
        pass

In [747]:
def get_pages_and_product_count():
    product_count = driver.find_element(By.XPATH, "//div[contains(@class, 'base-heading base-heading--h2 product-listing-viewer__headline')]//span[@class='base-heading__post']").text
    product_count=int(re.sub(r'[^\d]+', '', product_count))
    
    if product_count>24:
        a = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//a[@class='base-pagination__count']")))]
        max_page = int(a[-1].strip())
    else: max_page=0

    return max_page ,product_count

In [745]:
# get all the Categories & Subcategory name
driver.get("https://new.aldi.us/")
driver.find_element(By.CLASS_NAME, "base-page-header__burger-icon").click()
WebDriverWait(driver, 20).until(EC.element_to_be_clickable((By.XPATH, "//button[contains(@class, 'linklist-item flyout-submenu__item linklist-item--standard')]//*[contains(.,'Products')]/.."))).click()

Cat_subcat_list = []
Cat_subcat_url_list = []
Cat = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//div[@class='base-bordered-list__content']//span[@class='linklist-item__text']")))]
Cat_name = [unescape(i).strip() for i in Cat]
Cat_url = ['-'.join(unescape(i).strip().replace('&', '').lower().split()) for i in Cat]
Cat_dict = {Cat_name[i]:Cat_url[i] for i in range(len(Cat_name))}

for cat_name,cat_url in Cat_dict.items():
    xpath_subcat = "//button[contains(@class, 'linklist-item flyout-submenu__item linklist-item--standard')]//*[contains(.,'"+cat_name+"')]/.."
    button = driver.find_element(By.XPATH,xpath_subcat)
    button.click()
    Subcat = [my_elem.text for my_elem in driver.find_elements(By.XPATH, "//div[@class='base-bordered-list__content']//div[@class='linklist-item__text']") if my_elem.text!='']
    Subcat_url = ['-'.join(i.replace('&', '').lower().split()) for i in Subcat]
    Subcat_dict = {Subcat[i]:Subcat_url[i] for i in range(len(Subcat))}
    Cat_subcat_list.append({cat_name:Subcat})
    Cat_subcat_url_list.append({cat_url:Subcat_url})
    driver.find_element(By.XPATH, "//div[@class='base-flyout__sub-navi-headline']//span[@class='base-icon color--black']").click()


driver.find_element(By.XPATH, "//button[contains(@class, 'base-flyout__close')]").click()

In [762]:
######## Scraping for each SUBCATEGORY
### Create list to store item info
item_names=[]
item_brand_names=[]
item_proportions=[]
item_prices=[]
item_details=[]

### Navigate to Subcategory product page
driver.get(Product_Page+cat_key+'/yogurt-sour-cream')

### Get Subcategory product cnt and max page
product_count = driver.find_element(By.XPATH, "//div[contains(@class, 'base-heading base-heading--h2 product-listing-viewer__headline')]//span[@class='base-heading__post']").text
product_count=int(re.sub(r'[^\d]+', '', product_count))
if product_count>24:
    a = [my_elem.get_attribute("innerHTML") for my_elem in WebDriverWait(driver, 20).until(EC.visibility_of_all_elements_located((By.XPATH, "//a[@class='base-pagination__count']")))]
    max_page = int(a[-1].strip())
else: max_page = 0
#print(max_page,product_count)

### Iterate between each page and products on page
for i in range(1,max_page+1): 
    time.sleep(5)
    # when not in page 1, need to click to navigate to the page
    if i!=1:
        xpath = "//div[@class='product-listing-viewer__pagination']//a[contains(@aria-label,'page "+ str(i) + "')]"
        ignore_error_when_click(driver.find_element(By.XPATH,xpath))
        time.sleep(10)
    # count element on page to iterate over products on page
    element_cnt = len(driver.find_elements(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]"))
    
    for j in range(element_cnt):
        # each time element may be difference, need to find_element for each time and navigate to the product wanted to scrape
        element = driver.find_elements(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]")
        ignore_error_when_click(WebDriverWait(driver, 30).until(EC.element_to_be_clickable(element[j])))
        driver.implicitly_wait(10) 
        
        item_names.append(check_xpath_exist("//h1[contains(@class, 'product-details__title')]",'text',''))
        item_brand_names.append(check_xpath_exist("//a[contains(@class, 'base-link product-details__brand-name aem-link--text-link')]",'text',''))
        item_proportions.append(check_xpath_exist("//h2[contains(@class, 'product-details__unit-of-measurement')]",'text',''))
        item_prices.append(check_xpath_exist("//div[contains(@class, 'base-price price-section base-price--product-details')]//span[contains(@class,'price')]",'text',''))
        item_details.append(check_xpath_exist("//div[@class='show-more product-details__description']//div[contains(@class,'base-richtext')]",'text',''))
        
        # back to product page
        driver.get(Product_Page+cat_key+'/yogurt-sour-cream')
        if i!=1:
            xpath = "//div[@class='product-listing-viewer__pagination']//a[contains(@aria-label,'page "+ str(i) + "')]"
            ignore_error_when_click(driver.find_element(By.XPATH,xpath))
            time.sleep(10)
        
        # wait for website to load
        driver.implicitly_wait(10)
        time.sleep(2)

## Create Pandas table for each subcategory 
lst = list(zip(item_names,item_brand_names,item_proportions,item_prices,item_details))
pd.DataFrame(lst, columns =['Product Name', 'Brand', 'Proportions','price','details'], dtype = float) 

3 59


/var/folders/_7/j6gsg7kd1vxgq31_w637bnh00000gn/T/ipykernel_12181/2770128718.py:57: FutureWarning: Could not cast to float64, falling back to object. This behavior is deprecated. In a future version, when a dtype is passed to 'DataFrame', either all columns will be cast to that dtype, or a TypeError will be raised.
  pd.DataFrame(lst, columns =['Product Name', 'Brand', 'Proportions','price','details'], dtype = float)


,Product Name,Brand,Proportions,price,details
0,"Lowfat Unsweetened Plain Kefir, 32 oz",LIFEWAY,32 fl oz,$3.65,
1,"Lowfat Strawberry Kefir, 32 oz",LIFEWAY,32 fl oz,$3.65,
2,"Lowfat Blueberry Kefir, 32 oz",LIFEWAY,32 fl oz,$3.65,
3,"Danimals Strawberry Smoothies, 6 count",DANNON,18.6 fl oz,$3.29,
4,"Strawberry Flavor Greek Yogurt, 5.3 oz",TWO GOOD,5.3 oz,$1.45,
5,"Vanilla Flavor Greek Yogurt, 5.3 oz",TWO GOOD,5.3 oz,$1.45,
6,"Organic Whole Milk Vanilla Yogurt, 32 oz",SIMPLY NATURE,32 oz,$4.19,
7,"Organic Whole Milk Plain Yogurt, 32 oz",SIMPLY NATURE,32 oz,$4.19,
8,"Oreo and M&M Yogurt Multipack, 8 count",YOCRUNCH,8 x 6 oz,$7.85,Shop for an 8-pack of YoCrunch Oreo and M&M Yo...
9,"Lowfat Yogurt Tubes, 8 count",TEENAGE MUTANT NINJA TURTLES,16 oz,$1.29,


In [701]:
##### MAIN (WIP)
# # Go to each Category
# Main_Page = "https://new.aldi.us/"
# Product_Page = "https://new.aldi.us/products/"
# driver.get(Main_Page)
# cnt = 0
# for cat in Cat_subcat_url_list:
#     cnt+=1
#     cat_key,subcat_value = list(cat.items())[0]
#     driver.get(Product_Page+cat_key)

#     for subcat in subcat_value:
#         driver.get(Product_Page+cat_key+'/'+subcat)
#         time.sleep(2)
#         page, product_count = get_pages_and_product_count()
#         print(subcat,page,product_count)
#         time.sleep(1)
        
#         item_names=[]
#         item_brand_names=[]
#         item_proportions=[]
#         item_prices=[]
#         item_details=[]
#         cnt=0
#         driver.get(Product_Page+cat_key+'/'+subcat)
#         for i in range(1,max_page+1): 
#             if i !=1 then:
#                 driver.find_element(By.XPATH,"//div[@class='product-listing-viewer__pagination']//a[contains(@aria-label,'Next')]").click()
#             time.sleep(10)

#             for j in range(0,24):
#                 a = driver.find_elements(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]")
#                 #a[i].click()
#                 WebDriverWait(driver, 30).until(EC.element_to_be_clickable(a[j])).click()
#                 #driver.execute_script("arguments[0].click();", a[i])
#                 driver.implicitly_wait(10) 
#                 item_names.append(check_xpath_exist("//h1[contains(@class, 'product-details__title')]",'text',''))
#                 item_brand_names.append(check_xpath_exist("//a[contains(@class, 'base-link product-details__brand-name aem-link--text-link')]",'text',''))
#                 item_proportions.append(check_xpath_exist("//h2[contains(@class, 'product-details__unit-of-measurement')]",'text',''))
#                 item_prices.append(check_xpath_exist("//div[contains(@class, 'base-price price-section base-price--product-details')]//span[contains(@class,'price')]",'text',''))
#                 item_details.append(check_xpath_exist("//div[@class='show-more product-details__description']//div[contains(@class,'base-richtext')]",'text',''))
#                 driver.get(Product_Page+cat_key+'/yogurt-sour-cream?pages='+str(i))
#                 driver.implicitly_wait(10)
#                 time.sleep(2)
        
    
#     time.sleep(1)
    
#     if cnt==1: break

milk-milk-substitutes 2 45
eggs 0 6
cheese 9 198
butter 0 17
yogurt-sour-cream 3 59
coffee-creamer 0 24


In [539]:
# ## Click and Go to first Category = Diary and Eggs

# ### click the menu bar (when screen is small)
# button = driver.find_element(By.CLASS_NAME, "base-page-header__burger-icon")
# button.click()
# #driver.execute_script("arguments[0].click();", button)
# time.sleep(3)

# ### click Product
# button = driver.find_element(By.XPATH, "//button[contains(@class, 'linklist-item flyout-submenu__item linklist-item--standard')]//*[contains(.,'Products')]/..")
# button.click()
# #driver.execute_script("arguments[0].click();", button)
# time.sleep(3)

# ### click the "Diary & Eggs"
# button = driver.find_element(By.XPATH, "//button[contains(@class, 'linklist-item flyout-submenu__item linklist-item--standard')]//*[contains(.,'Dairy & Eggs')]/..")
# button.click()
# #driver.execute_script("arguments[0].click();", button)
# time.sleep(3)

# ### click Show all Diary & Eggs
# button = driver.find_element(By.XPATH, "//div[contains(@class, 'category-teaser__call-to-action')]//*[contains(.,'Show all Dairy & Eggs')]/..")
# button.click()
# #driver.execute_script("arguments[0].click();", button)


In [430]:
# driver.find_element(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]").click()
# item_name =  driver.find_element(By.XPATH, "//h1[contains(@class, 'product-details__title')]").text
# brand_name = driver.find_element(By.XPATH, "//a[contains(@class, 'base-link product-details__brand-name aem-link--text-link')]").text
# proportion = driver.find_element(By.XPATH, "//h2[contains(@class, 'product-details__unit-of-measurement')]").text
# price = driver.find_element(By.XPATH, "//div[contains(@class, 'base-price price-section base-price--product-details')]//span[contains(@class,'price')]").text
# product_details = driver.find_element(By.XPATH, "//div[@class='show-more product-details__description']").text
# product_details

In [410]:
# a = driver.find_elements(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]")
# b = driver.find_element(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]")
# #a[0],driver.find_element(By.XPATH, "//div[contains(@class, 'product-teaser-item product-grid__item')]")
# #b.click()
# driver.execute_script("arguments[0].click();", a[0])

In [429]:
# def wait_longer_if_not_clickable(elements):
#     try:
#         elements.click()
#     except ElementClickInterceptedException:
#         time.sleep(10)
#         return elements
#     return elements